In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\AISW-203-115\AppData\Roaming\nltk_data...


True

In [8]:
sia = SentimentIntensityAnalyzer()

col_text_list = ['testtext']
predict_text_list = ["I really hate you.",
                     "I don't like you.",
                     "I'm a person.",
                     "I like you.",
                     "I love you so much.",
                     "i really love soooooooo much",
                     "fuck you, you're a monster",
                     "snow falling slowly from sky",
                     "killing me softly with your voice"]

score_test = pd.DataFrame(predict_text_list, columns=col_text_list)

for index, row in score_test.iterrows():
    print(sia.polarity_scores(row['testtext']))
    print(sia.polarity_scores(row['testtext'])['compound'])
    print(row['testtext'])

{'neg': 0.666, 'neu': 0.334, 'pos': 0.0, 'compound': -0.6115}
-0.6115
I really hate you.
{'neg': 0.513, 'neu': 0.487, 'pos': 0.0, 'compound': -0.2755}
-0.2755
I don't like you.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0.0
I'm a person.
{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}
0.3612
I like you.
{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.6369}
0.6369
I love you so much.
{'neg': 0.0, 'neu': 0.4, 'pos': 0.6, 'compound': 0.6697}
0.6697
i really love soooooooo much
{'neg': 0.538, 'neu': 0.462, 'pos': 0.0, 'compound': -0.5423}
-0.5423
fuck you, you're a monster
{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.1531}
-0.1531
snow falling slowly from sky
{'neg': 0.468, 'neu': 0.532, 'pos': 0.0, 'compound': -0.6597}
-0.6597
killing me softly with your voice


복합점수를 기준으로 문장을 <br>
*강한부정*, *약한부정*, *중립*, *약한긍정*, *강한긍정* <br>
이렇게 5종류로 분류

* 강한부정 -2 : compound < -0.5
* 약한부정 -1 : -0.5 <= compound < 0
* 중립      0 : compound == 0
* 약한긍정  1 : 0 <= compound < 0.5
* 강한긍정  2 : 0.5 < compound

BeautifulSoup을 이용해 텍스트의 형태로 뉴스의 헤드라인을 가져옴 <br>
헤드라인에 두 가지 칼럼을 더 추가하기 위해서 pandas의 DataFrame을 사용
<br>
뉴스 헤드라인뿐만 아니라 헤더에 복합 점수, 평가(강한 부정 to 강한 긍정)를 추가

In [ ]:
import requests
from bs4 import BeautifulSoup

def getNews():
    URL = 'https://www.malaymail.com'
    page = requests.get(URL)
    soup = BeautifulSoup(page.text, 'html.parser')
    news_stories = soup.find_all('h2')
    news_no = 0
    
    col_list = ['score', 'predict', 'newstitle']
    news_list = []
    
    for new_story in news_stories:
        if news_no != 0:
            news_list.append(['', '', news_story.text.strip()])
        news_no = news_no + 1
        if news_no > 50:
            break
    return pd.DataFrame(news_list, columns=col_list)

뉴스 헤더라인, Vader로 감정 평가하기

In [3]:
import web_scraper_news as scraper
from nltk.sentiment import SentimentIntensityAnalyzer

def detScore(num):  # 강한부정부터 강한긍정으로 분류
    if num < -0.5: 
        return -2
    elif num >= -0.5 and num < 0:  
        return -1
    elif num == 0.0: 
        return 0
    elif num > 0 and num <= 0.5: 
        return 1
    else: 
        return 2
    
sia = SentimentIntensityAnalyzer()
news_df = scraper.get_news()

# for index, row in news_df.iterrows():
#     row['score'] = sia.polarity_scores(row['newstitle'])['compound']
    
news_df['score'] = news_df['newstitle'].apply(lambda title: sia.polarity_scores(title)['compound'])
    
news_df['predict'] = news_df['score'].apply(detScore)

In [4]:
news_df

,news_no,newstitle,score,predict
0,1,Israel losing global support over Gaza bombing...,0.0258,1
1,2,Cricket Australia warns Khawaja over Gaza message,-0.1027,-1
2,3,Brooklyn Nine-Nine star Andre Braugher dies ag...,0.0000,0
3,4,Thai MP jailed for posts insulting the monarchy,-0.7506,-2
4,5,BTS go into the army - what now for K-pop's ki...,0.0000,0
5,6,The Netflix show users spent 812m hours watching,0.0000,0
6,7,UN vote overwhelmingly demands Gaza ceasefire,-0.1280,-1
7,8,Argentina begins 'shock therapy' by devaluing ...,-0.3818,-1
8,9,MoD fined after email blunder risked Afghan lives,-0.2263,-1
9,10,Twenty-three killed in attack on Pakistan army...,-0.8225,-2
